# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 26/02/2024

In [ ]:
# USER: Change the path below to point to your local copy of ImageD11:

import os

home_dir = !echo $HOME
home_dir = str(home_dir[0])

# USER: You can change this location if you want

id11_code_path = os.path.join(home_dir, "Code/ImageD11")

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import matplotlib
import h5py
from tqdm.notebook import tqdm

%matplotlib widget
from matplotlib import pyplot as plt

import ImageD11.nbGui.nb_utils as utils

import ImageD11.grain
import ImageD11.unitcell
import ImageD11.indexing
import ImageD11.columnfile
import ImageD11.refinegrains
import ImageD11.sinograms.dataset
import ImageD11.sinograms.properties
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label

from ImageD11.blobcorrector import eiger_spatial

In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/data/visitor/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/James/nb_testing"

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_250um"

In [ ]:
# desination of H5 files

dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

print(ds)
print(ds.shape)

In [ ]:
# load 4d peaks from file

cf_4d = ImageD11.columnfile.colfile_from_hdf(ds.col4dfile)

major_phase_par_file = ds.parfile

cf_4d.parameters.loadparameters(ds.parfile)
cf_4d.updateGeometry()

In [ ]:
# plot the 4D peaks (fewer of them) as a cake (two-theta vs eta)
# if the parameters in the par file are good, these should look like straight lines

fig, ax = plt.subplots()

ax.scatter(cf_4d.ds, cf_4d.eta, s=1)

ax.set_xlabel("dstar")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# isolate main phase peaks, and remove them from the dataset

major_phase_cf_dstol = 0.0075

major_phase_peaks_mask = utils.unitcell_peaks_mask(cf_4d, dstol=major_phase_cf_dstol, dsmax=cf_4d.ds.max())

minor_phase_peaks = cf_4d.copy()
minor_phase_peaks.filter(~major_phase_peaks_mask)

In [ ]:
# plot the remaining peaks as a cake (d-star vs eta)

fig, ax = plt.subplots()

ax.scatter(minor_phase_peaks.ds, minor_phase_peaks.eta, s=1)

ax.set_xlabel("dstar")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# Update geometry for minor phase peaks

minor_phase_par_file = os.path.join(processed_data_root_dir, '../../../SCRIPTS/James/S3DXRD/Au.par')

ds.parfile = minor_phase_par_file

minor_phase_peaks.parameters.loadparameters(ds.parfile)
minor_phase_peaks.updateGeometry()

cf_strong_frac = 0.92
cf_strong_dsmax = 0.9
cf_strong_dstol = 0.005

cf_strong = utils.selectpeaks(minor_phase_peaks, dstol=cf_strong_dstol, dsmax=cf_strong_dsmax, frac=cf_strong_frac, doplot=0.5)
print(cf_strong.nrows)

In [ ]:
# OPTIONAL: export CF to an flt so we can play with it with ImageD11_gui
# uncomment the below line

# cf_4d.writefile(f'{sample}_{dataset}_4d_peaks.flt')

In [ ]:
# now we can take a look at the intensities of the remaining peaks

fig, ax = plt.subplots()

ax.plot(cf_strong.ds, cf_strong.sum_intensity,',')
ax.semilogy()

ax.set_xlabel("dstar")
ax.set_ylabel("Intensity")

plt.show()

In [ ]:
# now we can define unit cells from our parameters

minor_phase_unitcell = ImageD11.unitcell.unitcell_from_parameters(cf_strong.parameters)
minor_phase_unitcell.makerings(cf_strong.ds.max())

major_phase_unitcell = ImageD11.unitcell.unitcell_from_parameters(cf_4d.parameters)
major_phase_unitcell.makerings(cf_strong.ds.max())

In [ ]:
# now let's plot our peaks again, with the rings from the unitcell included, to check our lattice parameters are good

fig, ax = plt.subplots()

skip=1
ax.scatter( cf_strong.ds[::skip], cf_strong.eta[::skip],s=0.5)
ax.plot( minor_phase_unitcell.ringds, [0,]*len(minor_phase_unitcell.ringds), '|', ms=90, c="red", label="Minor phase")
ax.plot( major_phase_unitcell.ringds, [0,]*len(major_phase_unitcell.ringds), '|', ms=90, c="green", label="Major phase")
ax.set_xlabel('1 / d ($\AA$)')
ax.set_ylabel('$\\eta$ (deg)')

lgnd = ax.legend(loc="upper right", scatterpoints=1, fontsize=10)
for handle in lgnd.legend_handles:
    handle.set(markersize=6.0)

plt.show()

In [ ]:
# specify our ImageD11 indexer with these peaks

indexer = ImageD11.indexing.indexer_from_colfile(cf_strong)

print(f"Indexing {cf_strong.nrows} peaks")

In [ ]:
# USER: set a tolerance in d-space (for assigning peaks to powder rings)

indexer_ds_tol = 0.005
indexer.ds_tol = indexer_ds_tol

# change the log level so we can see what the ring assigments look like

ImageD11.indexing.loglevel = 1

# assign peaks to powder rings

indexer.assigntorings()

# change log level back again

ImageD11.indexing.loglevel = 3

In [ ]:
# let's plot the assigned peaks

fig, ax = plt.subplots()

# indexer.ra is the ring assignments

ax.scatter(cf_strong.ds, cf_strong.eta, c=indexer.ra, cmap='tab20', s=1)
ax.set_xlabel("d-star")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# now we are indexing!
# we have to choose which rings we want to generate orientations on
# generally we want two or three low-multiplicity rings that are isolated from other phases
# take a look at the ring assignment output from a few cells above, and choose two or three
rings_for_gen = [0, 4]

# now we want to decide which rings to score our found orientations against
# generally we can just exclude dodgy rings (close to other phases, only a few peaks in etc)
rings_for_scoring = [0, 3, 4]

# the sequence of hkl tolerances the indexer will iterate through
hkl_tols_seq = [0.01, 0.02, 0.03, 0.04, 0.05]
# the sequence of minpks fractions the indexer will iterate through
fracs = [0.9]
# the tolerance in g-vector angle
cosine_tol = np.cos(np.radians(90 - 0.25))
# the max number of UBIs we can find per pair of rings
max_grains = 1000

grains, indexer = utils.do_index(cf=cf_strong,
                                dstol=indexer_ds_tol,
                                forgen=rings_for_gen,
                                foridx=rings_for_scoring,
                                hkl_tols=hkl_tols_seq,
                                fracs=fracs,
                                cosine_tol=cosine_tol,
                                max_grains=max_grains
)

In [ ]:
# set grain GIDs (useful if we ever delete a grain)
for i, g in enumerate(grains):
    g.gid = i
    
    g.a = np.cbrt(np.linalg.det(g.ubi))

In [ ]:
mean_unit_cell_lengths = [grain.a for grain in grains]

fig, ax = plt.subplots()
ax.plot(mean_unit_cell_lengths)
ax.set_xlabel("Grain ID")
ax.set_ylabel("Unit cell length")
plt.show()

a0 = np.median(mean_unit_cell_lengths)
    
print(a0)

In [ ]:
# assign peaks to grains

peak_assign_tol = 0.05

utils.assign_peaks_to_grains(grains, cf_strong, tol=peak_assign_tol)

print("Storing peak data in grains")
# iterate through all the grains
for g in tqdm(grains):
    # store this grain's peak indices so we know which 4D peaks we used for indexing
    g.mask_4d = cf_strong.grain_id == g.gid
    g.peaks_4d = cf_strong.index[cf_strong.grain_id == g.gid]

In [ ]:
utils.plot_index_results(indexer, cf_strong, 'First attempt')

In [ ]:
utils.plot_grain_sinograms(grains, cf_strong, 25)

In [ ]:
# save grain data

phase_name = 'Au'

utils.save_s3dxrd_grains_minor_phase_after_indexing(grains, ds, phase_name)

In [ ]:
ds.parfile = major_phase_par_file
ds.save()

In [ ]:
if 1:
    raise ValueError("Change the 1 above to 0 to allow 'Run all cells' in the notebook")

In [ ]:
# Now that we're happy with our indexing parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_-50um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]
    
samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_200um"]}
    
# now we have our samples_dict, we can process our data:

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        if not os.path.exists(dset_path):
            print(f"Missing DataSet file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        ds.grainsfile_minor_phase = os.path.join(ds.analysispath, ds.dsname + f'_grains_{phase_name}.h5')
        
        if os.path.exists(ds.grainsfile_minor_phase):
            print(f"Already have grains for {dataset} in sample {sample}, skipping")
            continue
        
        cf_4d = ImageD11.columnfile.colfile_from_hdf(ds.col4dfile)
        
        major_phase_par_file = ds.parfile
        
        cf_4d.parameters.loadparameters(ds.parfile)
        cf_4d.updateGeometry()

        major_phase_peaks_mask = utils.unitcell_peaks_mask(cf_4d, dstol=major_phase_cf_dstol, dsmax=cf_4d.ds.max())

        minor_phase_peaks = cf_4d.copy()
        minor_phase_peaks.filter(~major_phase_peaks_mask)
        
        ds.parfile = minor_phase_par_file
        minor_phase_peaks.parameters.loadparameters(ds.parfile)
        minor_phase_peaks.updateGeometry()

        cf_strong = utils.selectpeaks(minor_phase_peaks, dstol=cf_strong_dstol, dsmax=cf_strong_dsmax, frac=cf_strong_frac)

        grains, indexer = utils.do_index(cf=cf_strong,
                                dstol=indexer_ds_tol,
                                forgen=rings_for_gen,
                                foridx=rings_for_scoring,
                                hkl_tols=hkl_tols_seq,
                                fracs=fracs,
                                cosine_tol=cosine_tol,
                                max_grains=max_grains
        )
        
        for i, g in enumerate(grains):
            g.gid = i
            
        utils.assign_peaks_to_grains(grains, cf_strong, tol=peak_assign_tol)

        print("Storing peak data in grains")
        for g in tqdm(grains):
            g.mask_4d = cf_strong.grain_id == g.gid
            g.peaks_4d = cf_strong.index[g.mask_4d]
            
        utils.save_s3dxrd_grains_minor_phase_after_indexing(grains, ds, phase_name)
        
        ds.parfile = major_phase_par_file
        ds.save()

print("Done!")